# AIDAR Objective 1.0 Hypotheses Testing
* Query a representative sample of the relevant dataset for each hypotheses
* Test the null and alternative hypotheses (P-value) for the confidence interval 
* Present the results for discussion and evaluation

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


| Analytics Activity |Hypothese to test | Required data |
|--------|-----|--------------|
|AA-1.1.1.1 |Test whether Adsets with higher budget_remaining correlate with higher impressions but not necessarily with better ROI. |client type<br>ad_id<br>timestamp<br>remaining budget (allocated - spent)<br>impressions count<br>ROI (CTR, CPM, ...) |
|AA-1.1.1.2 |Discover all  other correlating factors that may contribute to underperforming adsets | |
|AA-1.2.1.1 |Consider all optimization_goal types (e.g., conversions) and their Powerset combinations on Adsets that outperform those targeting general engagement. | |
|AA-1.3.1.1 |Campaigns with defined bid_strategy types (e.g., cost cap) lead to lower CPA. | |
|AA-1.3.1.2 |Look at the statistically significant power sets of objective and strategy elements | |
|AA-1.4.1.1 |Test whether campaigns with a higher daily_budget allocated to peak hours achieve better ROI | |
|AA-1.4.1.2 |Calculate the timeframes that offer high impact time segments or seasonal effects (e.g. peak hours) | |
|AA-1.5.1.1 |Evaluate whether Ads targeting specific demographics yield better ROI than generic targeting. | |

## Instantiate Classes

In [2]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('mining/')[0])
from rezaware.modules.etl.loader import sparkRDBM as db
# from rezaware.modules.etl.loader import __propAttr__ as attr

''' restart initiate classes '''
if debug:
    import importlib
    db = importlib.reload(db)
    # attr=importlib.reload(attr)

__desc__ = "read and write BigQuery dataset for hypothese testing"
clsSDB = db.dataWorkLoads(
    desc=__desc__,
    db_type = 'bigquery',
    db_driver=None,
    db_hostIP=None,
    db_port = None,
    db_name = None,
    db_schema='combined_data_facebook_ads',
    spark_partitions=None,
    spark_format = 'bigquery',
    spark_save_mode=None,
    # spark_jar_dir = _jar,
)
# if clsSDB.session:
#     clsSDB._session.stop
print("\n%s class initialization and load complete!" % __desc__)

All functional __PROPATTR__-libraries in LOADER-package of ETL-module imported successfully!
All functional SPARKRDBM-libraries in LOADER-package of ETL-module imported successfully!
All functional SPARKRDBM-libraries in LOADER-package of ETL-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
__propAttr__ Class initialization complete

read and write BigQuery dataset for hypothese testing class initialization and load complete!


## Load data
There are two options for loading the data
1. querying with a select statement by defining the sql _query_ statement string
2. reading an entire table with row limit by defining the _tbl_ comprising _project:dataset.table_

By default the function _read_data_from_table() will check for the select variable; else will revert to the _tbl description to read the data
Use either or; i.e.,: 
* if using the select query set ```db_table = None``` and assign ```select=query```
* if using db_table, then set ```select = None``` and assign ```db_table = _tbl```

In [4]:
query = """
    SELECT id, name, status, campaign, end_time, estimate, dsa_payor, targeting,
        account_id, bid_amount, start_time, campaign_id, created_time, daily_budget,
        source_adset, updated_time, billing_event, lifetime_imps, dsa_beneficiary, 
        lifetime_budget, promoted_object, review_feedback, source_adset_id, budget_remaining,
        destination_type, effective_status, configured_status, optimization_goal,
        use_new_app_click, is_dynamic_creative, campaign_active_time, campaign_attribution,
        optimization_sub_event, recurring_budget_semantics, min_budget_spend_percentage,
        multi_optimization_goal_weight
    FROM fresh-deck-445306-b0.combined_data_facebook_ads.combine__metadata_adset as adset
    LIMIT 1000
"""
_tbl = "fresh-deck-445306-b0:combined_data_facebook_ads._bqc_fc44f128164342be8e2acc6b3c5587ba"
options = {
    "project" : 'fresh-deck-445306-b0',
    "dataset" : 'combined_data_facebook_ads',
}
sdf = clsSDB.read_data_from_table(
    select=None, #query,
    db_table=_tbl,
    **options
)
print("Loaded %d rows" % sdf.count())
sdf.printSchema()

Loaded 1000 rows
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- status: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- estimate: string (nullable = true)
 |-- dsa_payor: string (nullable = true)
 |-- targeting: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- bid_amount: double (nullable = true)
 |-- start_time: string (nullable = true)
 |-- campaign_id: string (nullable = true)
 |-- created_time: string (nullable = true)
 |-- daily_budget: double (nullable = true)
 |-- source_adset: string (nullable = true)
 |-- updated_time: string (nullable = true)
 |-- billing_event: string (nullable = true)
 |-- lifetime_imps: double (nullable = true)
 |-- dsa_beneficiary: string (nullable = true)
 |-- lifetime_budget: double (nullable = true)
 |-- promoted_object: string (nullable = true)
 |-- review_feedback: string (nullable = true)
 |-- source_adset_id: string (nullable = true

## Randomly sample the data
* _Tolerance_ interval: 90% (0.90 percentile) ensures that p% of the sample
* _Confidence_ interval: 95% (alpha=0.95) to estimate the statistical significance (P-value)
* _Prediction_ interval
* Define the _Population_ by ramdomly selecting N representative rows

In [9]:
def confidence_interval(df, grp):
    col = F.when(F.col('status') == F.lit('ACTIVE'), 1.0).otherwise(0.0)

    return df.groupBy(grp).agg(
        F.mean(col).alias('avg'),
        F.count(F.lit(1)).alias('total'),
        F.stddev(col).alias('std')
    ).withColumn(
        'ci95_hi',
        F.col('avg') + 1.96 * F.col('std') / F.sqrt(F.col('total'))
    ).withColumn(
        'ci95_lo',
        F.col('avg') - 1.96 * F.col('std') / F.sqrt(F.col('total'))
    )

confidence_interval(sdf, 'billing_event').show()

+-------------+-----+-----+------------------+-----------------+------------------+
|billing_event|  avg|total|               std|          ci95_hi|           ci95_lo|
+-------------+-----+-----+------------------+-----------------+------------------+
|  IMPRESSIONS|0.566| 1000|0.4958728565770563|0.596734518070095|0.5352654819299049|
+-------------+-----+-----+------------------+-----------------+------------------+



In [5]:
from pyspark.sql import functions as F

_filter_by_cols = ['id', 'campaign_id', 'status', 'effective_status', 'configured_status',
                   'billing_event', 'optimization_goal', 'start_time', 'end_time',
                   'bid_amount', 'daily_budget', 'lifetime_budget', 'budget_remaining', 'destination_type']

sample_sdf = sdf.select(_filter_by_cols).orderBy(F.rand()).limit(100)
print("randomly selected %d rows" % sample_sdf.count())
sample_sdf.show(n=3)

randomly selected 100 rows


[Stage 8:>                                                          (0 + 1) / 1]

+-----------------+-----------------+------+----------------+-----------------+-------------+-----------------+--------------------+--------------------+----------+------------+---------------+----------------+----------------+
|               id|      campaign_id|status|effective_status|configured_status|billing_event|optimization_goal|          start_time|            end_time|bid_amount|daily_budget|lifetime_budget|budget_remaining|destination_type|
+-----------------+-----------------+------+----------------+-----------------+-------------+-----------------+--------------------+--------------------+----------+------------+---------------+----------------+----------------+
|23849924654200708|23849924654160708|ACTIVE|          ACTIVE|           ACTIVE|  IMPRESSIONS|  POST_ENGAGEMENT|2022-01-09T10:00:...|2022-01-11T10:00:...|      null|        null|           null|             0.0|       UNDEFINED|
|    6296484390149|    6296375589549|PAUSED|          PAUSED|           PAUSED|  IMPRESS

## Chi-square test

In [13]:
import pandas as pd
from scipy.stats import chi2_contingency

chisqr_sdf = sdf.select('billing_event','budget_remaining')

contingency_table = chisqr_sdf.stat.crosstab("billing_event", "budget_remaining")
contingency_table.show()

#Converting spark contingency_table to pandas DataFrame
contingency_table_df = contingency_table.toPandas()
contingency_table_df = contingency_table_df.set_index('billing_event_budget_remaining')
# Perform the chi-square test
chi2, p_value, degrees_of_freedom, expected_frequencies = chi2_contingency(contingency_table_df)

# Print the results
print("Chi-Square Statistic:", chi2)
print("P-Value:", p_value)
print("Degrees of Freedom:", degrees_of_freedom)

print(" ")
print("Contingency Table:")
print(contingency_table_df)

print(" ")
print("Expected Frequencies:")
print(pd.DataFrame(expected_frequencies, index=contingency_table_df.index, columns=contingency_table_df.columns))

+------------------------------+---+------+
|billing_event_budget_remaining|0.0|6000.0|
+------------------------------+---+------+
|                   IMPRESSIONS|999|     1|
+------------------------------+---+------+

Chi-Square Statistic: 0.0
P-Value: 1.0
Degrees of Freedom: 0
 
Contingency Table:
                                0.0  6000.0
billing_event_budget_remaining             
IMPRESSIONS                     999       1
 
Expected Frequencies:
                                  0.0  6000.0
billing_event_budget_remaining               
IMPRESSIONS                     999.0     1.0
